#### Урок 2. Парсинг HTML. BeautifulSoup, MongoDB

1) Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы) с сайта superjob.ru и hh.ru. Приложение должно анализировать несколько страниц сайта(также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:

    *Наименование вакансии
    *Предлагаемую зарплату (отдельно мин. и отдельно макс.)
    *Ссылку на саму вакансию        
    *Сайт откуда собрана вакансия
    
По своему желанию можно добавить еще работодателя и расположение. Данная структура должна быть одинаковая для вакансий с обоих сайтов. Общий результат можно вывести с помощью dataFrame через pandas.

In [54]:
from bs4 import BeautifulSoup as bs
import requests
from pprint import pprint
import pandas as pd
import numpy as np
import time
import re


In [65]:
def hh_pars_page(hh_xml, data):
    ''' 
    Парсинг HH-lxml страницы и добавление данных в pd.data
    '''

    list_block = hh_xml.find_all('div',{"class":"vacancy-serp-item"})


    for div in list_block:
        vacancy = div.find('a', {'class': 'bloko-link HH-LinkModifier'}).text
        #print(vacancy)
        vacancy_link = div.find('a', {'class': 'bloko-link HH-LinkModifier'})['href']
        #print(vacancy_link)
        employer = div.find('a', {'data-qa':'vacancy-serp__vacancy-employer'}).text
        #print(employer)

        compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})

        if not compensation:
            compensation1 = np.NaN
            compensation2 = np.NaN
        else:
            compensation = compensation.text.replace('.', '').replace(' ','').replace('руб','')
           # print(compensation)
            if '-' in compensation:
                compensations = compensation.split('-')

                compensation1 = compensations[0] #.replace(r'\D', '').replace(r'\xa', '')
                compensation1 = re.sub(r'\D', '',compensation1)
                compensation2 = compensations[1] #.replace(r'\D', '').replace(r'\xa', '')
                compensation2 = re.sub(r'\D', '',compensation2)
                
                compensation1 = int(compensation1)
                compensation2 = int(compensation2)


            elif 'от' in compensation:
                compensation1 = compensation.replace('от','').replace(r'\xa', '')
                compensation1 = re.sub(r'\D', '',compensation1)
                compensation1 = int(compensation1)
                compensation2 = np.NaN
            elif 'до' in compensation:
                compensation1 = np.NaN
                compensation2 = compensation.replace('до','').replace(' ', '')
                compensation2 = re.sub(r'\D', '',compensation2)
                compensation2 = int(compensation2)

        #print(compensation1, compensation2)


        data = data.append({'vacancy':[vacancy], 'min_salary':[compensation1], 'max_salary':[compensation2], 'vacancy_link':[vacancy_link], 'link':['https://hh.ru'], 
                            'employer':[employer]}, ignore_index=True)

        #compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    return(data)
    

In [66]:
def sj_pars_page(sj_xml, data):
    ''' 
    Парсинг SJ-lxml страницы и добавление данных в pd.data
    '''

    list_block = sj_xml.find_all('div',{"class":"_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"})
    pprint(list_block)
    for div in list_block:
        vacancy = div.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).text
        #print(vacancy)
        vacancy_link = div.find('div', {'class': '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'}).findParent()['href']
        vacancy_link = 'https://www.superjob.ru' + vacancy_link                               
        #print(vacancy_link)
        employer = div.find('span', {'class':'_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI'}).text
        #print(employer)     

        compensation = div.find('span', {'class': '_3mfro _2Wp8I f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'}).text
        #print(compensation)
        compensation1 = np.NaN
        compensation2 = np.NaN
        if '—' in compensation:

            compensation1 = compensation.split('—')[0]
            compensation2 = compensation.split('—')[1]
            compensation1 = re.sub(r'\D', '',compensation1)
            compensation2 = re.sub(r'\D', '',compensation2)
            compensation1 = int(compensation1)
            compensation2 = int(compensation2)

        elif 'от' in compensation:
            compensation1 = compensation[3:]
            compensation1 = re.sub(r'\D', '',compensation1)
            #print("compensation1=", compensation1)
            compensation1 = int(compensation1)
            compensation2 = np.NaN
        elif 'договорённости' in compensation:
            compensation1 = np.NaN
            compensation2 = np.NaN
        elif 'до' in compensation:
            compensation1 = np.NaN
            compensation2 = compensation[3:]
            compensation2 = int(compensation2)
        
        
        data = data.append({'vacancy':[vacancy], 'min_salary':[compensation1], 'max_salary':[compensation2], 'vacancy_link':[vacancy_link], 'link':['https://www.superjob.ru'], 
                            'employer':[employer]}, ignore_index=True)

        #compensation = div.find('div', {'data-qa': 'vacancy-serp__vacancy-compensation'})
    return(data)
    

In [67]:
# Общие параметры поиска: текст вакансии и глубина
search_text = "методолог"
pages = 4
data = pd.DataFrame()

In [68]:
#  Обработка HH

main_link = ('https://hh.ru')
method = "/search/vacancy?area=1&st=searchVacancy&text="
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

url = main_link+method+search_text
print(main_link+method+search_text)

session = requests.Session()

for i in range(pages):
    html = session.get(url, headers=headers).text
    
    hh_xml = bs(html,'lxml')
    data = hh_pars_page(hh_xml, data)
    url = main_link + hh_xml.find('a', {'class': 'bloko-button HH-Pager-Controls-Next HH-Pager-Control'})['href']
    
    time.sleep(1)
    
    

https://hh.ru/search/vacancy?area=1&st=searchVacancy&text=методолог


In [69]:
#  Обработка SJ

main_link = ('https://www.superjob.ru')
method = "/vacancy/search/?keywords="
headers = {'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.120 Safari/537.36'}

url = main_link+method+search_text
print(main_link+method+search_text)

session = requests.Session()


for i in range(pages):
    html = session.get(url, headers=headers).text
    
    sj_xml = bs(html,'lxml')
    data = sj_pars_page(sj_xml, data)
    url = main_link + sj_xml.find('a', {'class': 'icMQ_ _1_Cht _3ze9n f-test-button-dalshe f-test-link-dalshe'})['href']
    
    time.sleep(1)


https://www.superjob.ru/vacancy/search/?keywords=методолог
[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"></div><div class="_2g1F-"><div><div class="_1Tocb"><a class="icMQ_ f-test-link-" href="/clients/piccasushivok-2535003.html" target="_self"><div class="lazyload-placeholder" style="height:44px"></div></a></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Specialist_po_razrabotke_kursov_Razrabotchik_distancionnyh_kursov_Metodolog _2JivQ _3dPok" href="/vakansii/specialist-po-razrabotke-kursov-32758228.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf"><span class="_1rS-s">Специалист</span> по разработке курсов / Разработчик дистанционных курсов / <span class="_1rS-s">Методолог</span></div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span cl

[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"></div><div class="_2g1F-"><div><div class="_1Tocb"></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Sistemnyj_analitik_(Senior) _2JivQ _3dPok" href="/vakansii/sistemnyj-analitik-32391171.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf"><span class="_1rS-s">Системный</span> <span class="_1rS-s">аналитик</span> (Senior)</div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span class="_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI"><a class="icMQ_ _205Zx f-test-link-Fabrikant Vm5jz" href="/clients/fabrikant-51336.html" target="_self">Фабрикант</a></span></div><div class="_2g1F-"><span class="_3mfro f-test-text-company-item-location _9fXTd _2JVkc _3e53o"><span class

[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"></div><div class="_2g1F-"><div><div class="_1Tocb"></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Metodist_VKK _2JivQ _3dPok" href="/vakansii/metodist-vkk-29229725.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf"><span class="_1rS-s">Методист</span> ВКК</div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span class="_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _3e53o _15msI"><a class="icMQ_ _205Zx f-test-link-Kolledzh_informatiki_i_programmirovaniya Vm5jz" href="/clients/kolledzh-informatiki-i-programmirovaniya-1992275.html" target="_self">Колледж информатики и программирования</a></span></div><div class="_2g1F-"><span class="_3mfro f-test-text-company-item-location _9fX

[<div class="_3zucV _2GPIV f-test-vacancy-item i6-sc _3VcZr"><div class="_36e54 _2ntFx"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_212By _37XTu"><div class="_2g1F-"><div class="_1_bQo _2FJA4 _1iZ5S"><div class="_2g1F-"><span class="_1kf9U _1VKE- f-test-badge _2FkY7">Отклик без резюме</span></div></div></div><div class="_2g1F-"><div><div class="_1Tocb"><a class="icMQ_ f-test-link-" href="/clients/sberbank-rossii-297921.html" target="_self"><div class="lazyload-placeholder" style="height:44px"></div></a></div><div class="_3syPg _1_bQo _2FJA4"><div class="_2g1F-"><a class="icMQ_ _1QIBo f-test-link-Java-razrabotchik_Cash_Management _2JivQ _3dPok" href="/vakansii/java-razrabotchik-cash-management-32671005.html" target="_blank"><div class="_3mfro CuJz5 PlM3e _2JVkc _3LJqf">Java-разработчик Cash Management</div></a></div><div class="_2g1F-"><div class="_2XXYS _2cxK3"><div class="_2g1F-"><span class="_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXT

In [70]:
data.head()

employer             link max_salary  \
0  [ Всероссийский Банк Развития Регионов (ВБРР)]  [https://hh.ru]      [nan]   
1                [ Ренессанс cтрахование, Группа]  [https://hh.ru]      [nan]   
2                [ Ренессанс cтрахование, Группа]  [https://hh.ru]      [nan]   
3                                      [ FinFort]  [https://hh.ru]      [nan]   
4         [ Ростелеком - Центры обработки данных]  [https://hh.ru]      [nan]   

  min_salary                                            vacancy  \
0      [nan]            [Методолог кадрового администрирования]   
1      [nan]  [Методолог/Руководитель направления методологи...   
2      [nan]  [Ведущий специалист/Риск-менеджер в Управление...   
3    [70000]                                        [Методолог]   
4      [nan]                                        [Методолог]   

                                        vacancy_link  
0  [https://hh.ru/vacancy/34240183?query=%D0%BC%D...  
1  [https://hh.ru/vacancy/34005950?query=%D0%BC%D...  
2  [https://hh.ru/vacancy/34195483?query=%D0%BC%D...  
3  [https://hh.ru/vacancy/33436906?query=%D0%BC%D...  
4  [https://hh.ru/vacancy/32082936?query=%D0%BC%D...

In [71]:
data.to_pickle('vacancy_pd.pkl')

In [72]:
data_import = pd.read_pickle('vacancy_pd.pkl')
data_import

employer  \
0       [ Всероссийский Банк Развития Регионов (ВБРР)]   
1                     [ Ренессанс cтрахование, Группа]   
2                     [ Ренессанс cтрахование, Группа]   
3                                           [ FinFort]   
4              [ Ростелеком - Центры обработки данных]   
5                                      [ООО СТРАТЕГИЯ]   
6           [ПАО «Группа Черкизово», Центральный офис]   
7                                     [АО БАНК ИНТЕЗА]   
8                   [ООО Озерная Горнорудная Компания]   
9                                        [ООО EveryCo]   
10                                         [ DM Basis]   
11                                 [ООО СанТехОптТорг]   
12                                       [ Ланта-Банк]   
13                                        [ Робокасса]   
14                                         [ООО Верме]   
15                              [ Группа компаний АСД]   
16   [ «РОСБАНК, Societe Generale Group», с опытом ...   
17                                            [ CSSSR]   
18                          [ООО ПОСМ Лоджистик Групп]   
19      [ Всероссийский Банк Развития Регионов (ВБРР)]   
20                                [ Информаудитсервис]   
21                               [ Брокеркредитсервис]   
22                                          [АО «ОСД»]   
23                                      [ Группа Гута]   
24                                            [АО ВРБ]   
25                                 [ РОСГОССТРАХ Банк]   
26                                    [ООО МСК АйАйСи]   
27                        [ Московский Кредитный Банк]   
28                                   [ООО Финстарлабс]   
29                            [ Банк Русский Стандарт]   
..                                                 ...   
130                                      [КДЦ № 2 ДЗМ]   
131  [ФГБУ ФГБУ ФНКЦ специализированных видов медиц...   
132  [ФГБУ ДПО Учебно-методический центр по образов...   
133                                 [Поликлиника ФНПР]   
134  [Филиал ФКУ «Налог-Сервис» ФНС России по ЦОД в...   
135  [ГБУЗ Психиатрическая клиническая больница №1 ...   
136                                              [АСТ]   
137                             [Правительство Москвы]   
138                                              [АСТ]   
139                                     [Благо Кредит]   
140                                  [Сбербанк России]   
141                                         [Тинькофф]   
142                                         [Медицина]   
143                                           [Юникод]   
144                                  [Сбербанк России]   
145                                  [Сбербанк России]   
146                                      [Перекресток]   
147                                        [СТРАТЕГИЯ]   
148                                   [АКБ «АВАНГАРД»]   
149                             [Правительство Москвы]   
150                                [Трест Гидромонтаж]   
151                             [КурьерСервисЭкспресс]   
152                             [МобильныеТелеСистемы]   
153                                            [Латек]   
154                   [Первая Экспедиционная Компания]   
155                                          [Изолюкс]   
156  [Всероссийский научно-исследовательский инстит...   
157                                    [ФГУ НПП Гамма]   
158                              [Группа компаний СНС]   
159                            [Кабинет ВОП в Реутове]   

                          link max_salary min_salary  \
0              [https://hh.ru]      [nan]      [nan]   
1              [https://hh.ru]      [nan]      [nan]   
2              [https://hh.ru]      [nan]      [nan]   
3              [https://hh.ru]      [nan]    [70000]   
4              [https://hh.ru]      [nan]      [nan]   
5              [https://hh.ru]   [150000]      [nan]   
6              [https://hh.ru]   [100000]      [nan]   
7

In [73]:
data_import.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 160 entries, 0 to 159
Data columns (total 6 columns):
employer        160 non-null object
link            160 non-null object
max_salary      160 non-null object
min_salary      160 non-null object
vacancy         160 non-null object
vacancy_link    160 non-null object
dtypes: object(6)
memory usage: 7.6+ KB
